In [ ]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from pinecone import Pinecone, ServerlessSpec
from operator import itemgetter

In [ ]:
# model = ChatGroq(model='llama-3.1-8b-instant',temperature=0)
model = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct',temperature=0)

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
pinecone_client = Pinecone()
index_name = "sistema-turismo-langchain"
index_name_exists = False

for index in pinecone_client.list_indexes():
  index_name = index['name']

  if index_name == "sistema-turismo-langchain":
    index_name_exists = True
    break

if not index_name_exists:
  pinecone_client.create_index(
    name=index_name,
    metric="cosine",
    dimension=384,
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
  )

In [ ]:
base_conhecimento_paris = "./base_conhecimento/paris.txt"
base_conhecimento_rio_de_janeiro = "./base_conhecimento/rio_de_janeiro.txt"

loader_paris = TextLoader(base_conhecimento_paris)
loader_rio = TextLoader(base_conhecimento_rio_de_janeiro)

documents_paris = loader_paris.load()
documents_rio = loader_rio.load()

all_documents = documents_paris + documents_rio

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200)

docs_split = text_splitter.split_documents(all_documents)

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
  documents=docs_split,
  embedding=embeddings_model,
  index_name=index_name
)

In [ ]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

In [ ]:
# Recebe a intenção do usuário e classifica em uma das categorias predefinidas
prompt_template_category_definition = PromptTemplate.from_template(
  """
    Classifique a pergunta do usuário em uma das seguintes categorias. Seja muito severo na classificação.
    Escreva apenas uma categoria. Por exemplo, se a pergunta for sobre roteiro de viagem, responda apenas "Roteiro de Viagem".
    Se a pergunta não se enquadrar em nenhuma das categorias, responda apenas "Geral".

    Categorias:
    - Roteiro de Viagem
    - Logística de Transporte
    - Informações sobre pontos turísticos
    - Guia de tradução e idiomas
    - Geral

    Pergunta: {query}
    Classificação:
  """
)

category_classification_chain = (prompt_template_category_definition | model | StrOutputParser())

In [ ]:
# Cadeia para extrair o perfil do usuário

prompt_extract_profile = PromptTemplate.from_template(
"""
  Analise a consulta do usuário e identifique o perfil do turista.
  Se não for possível identificar, retorne "geral".

  Perfis possíveis: aventura, cultural, gastronômico, relaxante, família.

  Consulta do usuário: {query}
  Perfil:
"""
)

profile_extraction_chain = prompt_extract_profile | model | StrOutputParser()

In [ ]:
guide_to_the_assistants = """
  Sua tarefa é responder à consulta do usuário baseando-se ÚNICA E EXCLUSIVAMENTE no contexto fornecido.
  É estritamente proibido utilizar qualquer conhecimento externo ao que foi apresentado no contexto.
  Não invente informações. Não complete detalhes que não estão no texto.
  Se a resposta para a pergunta não puder ser encontrada no contexto fornecido, você deve responder exatamente:
  "Desculpe, não tenho informações sobre isso na minha base de conhecimento atual."
  Não tente adivinhar ou inferir informações.
"""

In [ ]:
prompt_template_travel_itinerary = PromptTemplate(
  input_variables=["query", "profile", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer roteiros de viagem personalizados.
    Com base na consulta do usuário, crie um roteiro detalhado que inclua:
    - Principais atrações a serem visitadas
    - Eventos locais durante o período da viagem
    - Sugestões que se alinhem ao perfil do turista.

    Perfil do turista: {profile}
    Consulta do usuário: {query}
    Context: {context}
  """ % guide_to_the_assistants
)

In [ ]:
prompt_template_transport_logistics = PromptTemplate(
  input_variables=["query", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer informações sobre logística de transporte.
    Com base na consulta do usuário, apresente um relatório logístico que inclua:
    - Opções de transporte do local
    - Sugestões de acomodação
    - Dicas práticas para facilitar a viagem

    Consulta do usuário: {query}
    Context: {context}

  """ % guide_to_the_assistants
)

In [ ]:
prompt_template_local_information = PromptTemplate(
  input_variables=["query", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer informações locais detalhadas.
    Com base na consulta do usuário, forneça informações específicas sobre:
    - Pontos turísticos
    - Restaurantes recomendados
    - Horários de funcionamento e outras informações relevantes

    Consulta do usuário: {query}
    Context: {context}
    """ % guide_to_the_assistants
)

rag_chain = (
  {"context": retriever, "query": RunnablePassthrough()}
  | prompt_template_local_information
  | model
  | StrOutputParser()
)

In [ ]:
prompt_template_translation_guide = PromptTemplate(
  input_variables=["query"],
  template= """
    Você é um assistente especializado em fornecer guias de tradução para viajantes.
    Com base na consulta do usuário, crie um guia de tradução que inclua:
    - Frases úteis no idioma local
    - Dicas culturais para facilitar a comunicação

    Consulta do usuário: {query}
    """
)

In [ ]:
prompt_template_general = PromptTemplate(
  input_variables=["query"],
  template="""
    Guia geral: %s

    Você é um assistente de informações gerais.
    Se a consulta do usuário não se enquadrar nas categorias de roteiro de viagem,
    logística de transporte, informações locais ou guia de tradução,
    responda com uma mensagem educada informando que a consulta não pode ser atendida.
    E sugira tipos de perguntas que você pode responder.

    Consulta do usuário: {query}
    """ % (guide_to_the_assistants)
)

In [ ]:
travel_itinerary_chain = {
    "profile": itemgetter("query") | profile_extraction_chain,
    "context": itemgetter("query") | retriever,
    "query": itemgetter("query")
} | prompt_template_travel_itinerary

transport_logistics_chain = {
    "context": itemgetter("query") | retriever,
    "query": itemgetter("query")
} | prompt_template_transport_logistics

local_information_chain = {
    "context": itemgetter("query") | retriever,
    "query": itemgetter("query")
} | prompt_template_local_information

full_chain = RunnableBranch(
    (
        lambda x: "roteiro de viagem" in x["category"].lower(),
        travel_itinerary_chain
    ),
    (
        lambda x: "logística de transporte" in x["category"].lower(),
        transport_logistics_chain
    ),
    (
        lambda x: "informações sobre pontos turísticos" in x["category"].lower(),
        local_information_chain
    ),
    (
         lambda x: "guia de tradução" in x["category"].lower(),
         itemgetter("query") | prompt_template_translation_guide
    ),
    itemgetter("query") | prompt_template_general
)

final_orchestrator_chain = {
    "category": category_classification_chain,
    "query": itemgetter("query")
} | full_chain | model | StrOutputParser()

In [ ]:
# Exemplo 1: Roteiro de Viagem
query = "Estou planejando uma viagem gastronomica de 3 dias em Paris. Quais atrações você recomenda?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

In [ ]:
# Exemplo 2: Logística de Transporte
query = "Quais são as melhores opções de transporte do aeroporto Charles de Gaulle para o centro de Paris?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

In [ ]:
# Exemplo 3: Informações sobre pontos turísticos
query = "Quais são os pontos turisticos de paris?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

In [ ]:
# Exemplo 4: Guia de tradução e idiomas
query = "Como digo 'onde fica o banheiro?' em francês?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

In [ ]:
# Exemplo 5: Geral
query = "Qual é a capital da França?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)